In [22]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

In [23]:
categories = ['comp.graphics', 'sci.crypt', 'sci.electronics']
remove = ('headers', 'footers', 'quotes')

twenty_train_full = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42,
                                       remove=remove)
twenty_test_full = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42,
                                      remove=remove)

In [24]:
import nltk
from nltk import word_tokenize
from nltk.stem import *

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ruslan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
def stemming(data):
    porter_stemmer = PorterStemmer()
    stem = []
    for text in data:
        nltk_tokens = word_tokenize(text)
        line = ''.join([' ' + porter_stemmer.stem(word) for word in nltk_tokens])
        stem.append(line)
    return stem


stem_train = stemming(twenty_train_full.data)
stem_test = stemming(twenty_test_full.data)

Вариант 12
RF, SVM, LR

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [27]:
stop_words = [None, 'english']
max_features_values = [100, 500, 1000, 5000, 10000]
use_idf = [True, False]

rf_first = range(1, 5, 1)
rf_second = range(5, 100, 20)

rf_tree_max_depth = [*rf_first, *rf_second]

parameters_rf = {
    'vect__max_features': max_features_values,
    'vect__stop_words': stop_words,
    'tfidf__use_idf': use_idf,
    'clf__n_estimators': range(1, 10, 1),
    'clf__criterion': ('gini', 'entropy'),
    'clf__max_depth': rf_tree_max_depth,
}

parameters_lr = {
    'vect__max_features': max_features_values,
    'vect__stop_words': stop_words,
    'tfidf__use_idf': use_idf,
    'clf__solver': ['newton-cg', 'lbfgs', 'sag', 'liblinear'],
    'clf__penalty': ['l2']
}

parameters_lr_l1 = {
    'vect__max_features': max_features_values,
    'vect__stop_words': stop_words,
    'tfidf__use_idf': use_idf,
    'clf__solver': ['liblinear'],  # Используем только 'liblinear' для l1
    'clf__penalty': ['l1'],
}

parameters_svm = {
    'vect__max_features': max_features_values,
    'vect__stop_words': stop_words,
    'tfidf__use_idf': use_idf,
}

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

Random Forest without stemming

In [29]:
text_clf_rf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', RandomForestClassifier())])
gscv_rf = GridSearchCV(text_clf_rf, param_grid=parameters_rf, n_jobs=-1)
gscv_rf.fit(twenty_train_full.data, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'clf__criterion': ('gini', 'entropy'),
                         'clf__max_depth': [1, 2, 3, 4, 5, 25, 45, 65, 85],
                         'clf__n_estimators': range(1, 10),
                         'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

Random Formest with stemming

In [30]:
text_clf_rf_stem = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', RandomForestClassifier())])
gscv_rf_stem = GridSearchCV(text_clf_rf_stem, param_grid=parameters_rf, n_jobs=-1)
gscv_rf_stem.fit(stem_train, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'clf__criterion': ('gini', 'entropy'),
                         'clf__max_depth': [1, 2, 3, 4, 5, 25, 45, 65, 85],
                         'clf__n_estimators': range(1, 10),
                         'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

SVM without stem

In [31]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SVC())])
gscv_svm = GridSearchCV(text_clf_svm, param_grid=parameters_svm, n_jobs=-1)
gscv_svm.fit(twenty_train_full.data, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', SVC())]),
             n_jobs=-1,
             param_grid={'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

SVM with stem

In [32]:
text_clf_svm_stem = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', SVC(kernel='linear', C=1.0))])
gscv_svm_stem = GridSearchCV(text_clf_svm_stem, param_grid=parameters_svm, n_jobs=-1)
gscv_svm_stem.fit(stem_train, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', SVC(kernel='linear'))]),
             n_jobs=-1,
             param_grid={'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

Логистическая регрессия без стеминга

In [33]:
text_clf_lr = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', LogisticRegression())])
gscv_lr = GridSearchCV(text_clf_lr, param_grid=parameters_lr, n_jobs=-1)
gscv_lr.fit(twenty_train_full.data, twenty_train_full.target)

text_clf_lr_l1 = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', LogisticRegression())])
gscv_lr_l1 = GridSearchCV(text_clf_lr_l1, param_grid=parameters_lr_l1, n_jobs=-1)
gscv_lr_l1.fit(twenty_train_full.data, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'clf__penalty': ['l1'], 'clf__solver': ['liblinear'],
                         'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

Логистическая регрессия со стеммингом

In [45]:
text_clf_lr_stem = Pipeline([('vect', CountVectorizer()),
                             ('tfidf', TfidfTransformer()),
                             ('clf', LogisticRegression())])
gscv_lr_stem = GridSearchCV(text_clf_lr_stem, param_grid=parameters_lr, n_jobs=-1)
gscv_lr_stem.fit(stem_train, twenty_train_full.target)

text_clf_lr_l1_stem = Pipeline([('vect', CountVectorizer()),
                             ('tfidf', TfidfTransformer()),
                             ('clf', LogisticRegression())])
gscv_lr_l1_stem = GridSearchCV(text_clf_lr_l1_stem, param_grid=parameters_lr_l1, n_jobs=-1)
gscv_lr_l1_stem.fit(stem_train, twenty_train_full.target)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'clf__penalty': ['l1'], 'clf__solver': ['liblinear'],
                         'tfidf__use_idf': [True, False],
                         'vect__max_features': [100, 500, 1000, 5000, 10000],
                         'vect__stop_words': [None, 'english']})

Результаты

In [44]:
from sklearn.metrics import classification_report

predicted_rf = gscv_rf.predict(twenty_test_full.data)
print('Случайный лес (RF) без стемминга\n')
print(classification_report(twenty_test_full.target, predicted_rf, target_names=categories))
print(gscv_rf.best_params_)

predicted_rf_stem = gscv_rf_stem.predict(twenty_test_full.data)
print('Случайный лес (RF) со стеммингом\n')
print(classification_report(twenty_test_full.target, predicted_rf_stem, target_names=categories))
print(gscv_rf_stem.best_params_)

Случайный лес (RF) без стемминга

                 precision    recall  f1-score   support

  comp.graphics       0.76      0.79      0.77       389
      sci.crypt       0.86      0.70      0.77       396
sci.electronics       0.64      0.74      0.69       393

       accuracy                           0.74      1178
      macro avg       0.75      0.74      0.74      1178
   weighted avg       0.75      0.74      0.74      1178

{'clf__criterion': 'gini', 'clf__max_depth': 65, 'clf__n_estimators': 9, 'tfidf__use_idf': False, 'vect__max_features': 5000, 'vect__stop_words': 'english'}
Случайный лес (RF) со стеммингом

                 precision    recall  f1-score   support

  comp.graphics       0.67      0.65      0.66       389
      sci.crypt       0.87      0.58      0.69       396
sci.electronics       0.53      0.73      0.62       393

       accuracy                           0.65      1178
      macro avg       0.69      0.65      0.66      1178
   weighted avg       0.69   

In [48]:
predicted_lr_stem = gscv_lr_stem.predict(twenty_test_full.data)
print('Логистическая регрессия (LR) без стеминга\n')
print(classification_report(twenty_test_full.target, predicted_lr_stem, target_names=categories))
print(gscv_lr_stem.best_params_)

predicted_lr_l1_stem = gscv_lr_l1_stem.predict(twenty_test_full.data)
print('Логистическая регрессия_l1 (LR) со стеммингом\n')
print(classification_report(twenty_test_full.target, predicted_lr_l1_stem, target_names=categories))
print(gscv_lr_l1_stem.best_params_)

Логистическая регрессия (LR) без стеминга

                 precision    recall  f1-score   support

  comp.graphics       0.80      0.79      0.80       389
      sci.crypt       0.89      0.69      0.78       396
sci.electronics       0.67      0.82      0.74       393

       accuracy                           0.77      1178
      macro avg       0.79      0.77      0.77      1178
   weighted avg       0.79      0.77      0.77      1178

{'clf__penalty': 'l2', 'clf__solver': 'newton-cg', 'tfidf__use_idf': True, 'vect__max_features': 5000, 'vect__stop_words': 'english'}
Логистическая регрессия_l1 (LR) со стеммингом

                 precision    recall  f1-score   support

  comp.graphics       0.77      0.72      0.75       389
      sci.crypt       0.91      0.59      0.71       396
sci.electronics       0.58      0.82      0.68       393

       accuracy                           0.71      1178
      macro avg       0.75      0.71      0.71      1178
   weighted avg       0.75    

In [42]:
predicted_lr = gscv_lr.predict(twenty_test_full.data)
print('Логистическая регрессия (LR) без стемминга\n')
print(classification_report(twenty_test_full.target, predicted_lr, target_names=categories))
print(gscv_lr.best_params_)

predicted_lr_l1 = gscv_lr_l1.predict(twenty_test_full.data)
print('Логистическая регрессия_l1 (LR) без стемминга\n')
print(classification_report(twenty_test_full.target, predicted_lr_l1, target_names=categories))
print(gscv_lr_l1.best_params_)

Логистическая регрессия (LR) без стемминга

                 precision    recall  f1-score   support

  comp.graphics       0.82      0.87      0.85       389
      sci.crypt       0.90      0.76      0.83       396
sci.electronics       0.75      0.83      0.79       393

       accuracy                           0.82      1178
      macro avg       0.83      0.82      0.82      1178
   weighted avg       0.83      0.82      0.82      1178

{'clf__penalty': 'l2', 'clf__solver': 'newton-cg', 'tfidf__use_idf': True, 'vect__max_features': 5000, 'vect__stop_words': 'english'}
Логистическая регрессия_l1 (LR) без стемминга

                 precision    recall  f1-score   support

  comp.graphics       0.82      0.80      0.81       389
      sci.crypt       0.89      0.68      0.77       396
sci.electronics       0.65      0.81      0.72       393

       accuracy                           0.76      1178
      macro avg       0.78      0.76      0.77      1178
   weighted avg       0.78   

In [41]:
predicted_svm = gscv_svm.predict(twenty_test_full.data)
print('Метод опорных векторов (SVM) без стемминга\n')
print(classification_report(twenty_test_full.target, predicted_svm, target_names=categories))
print(gscv_svm.best_params_)

predicted_svm_stem = gscv_svm_stem.predict(twenty_test_full.data)
print('Метод опорных векторов (SVM) со стеммингом\n')
print(classification_report(twenty_test_full.target, predicted_svm_stem, target_names=categories))
print(gscv_svm_stem.best_params_)

Метод опорных векторов (SVM) без стемминга

                 precision    recall  f1-score   support

  comp.graphics       0.83      0.86      0.85       389
      sci.crypt       0.93      0.74      0.83       396
sci.electronics       0.74      0.86      0.79       393

       accuracy                           0.82      1178
      macro avg       0.83      0.82      0.82      1178
   weighted avg       0.83      0.82      0.82      1178

{'tfidf__use_idf': True, 'vect__max_features': 5000, 'vect__stop_words': 'english'}
Метод опорных векторов (SVM) со стеммингом

                 precision    recall  f1-score   support

  comp.graphics       0.79      0.81      0.80       389
      sci.crypt       0.87      0.67      0.76       396
sci.electronics       0.67      0.81      0.73       393

       accuracy                           0.76      1178
      macro avg       0.78      0.76      0.76      1178
   weighted avg       0.78      0.76      0.76      1178

{'tfidf__use_idf': True,

In [47]:
writer = pd.ExcelWriter('./../output/3lab_result.xlsx', engine='openpyxl')

# Случайный лес (RF) без стемминга
df1 = pd.DataFrame(classification_report(predicted_rf, twenty_test_full.target, output_dict=True))

# Случайный лес (RF) со стеммингом
df2 = pd.DataFrame(classification_report(predicted_rf_stem, twenty_test_full.target, output_dict=True))

# Логистическая регрессия (LR) без стемминга
df3 = pd.DataFrame(classification_report(predicted_lr, twenty_test_full.target, output_dict=True))

# Логистическая регрессия_l1 (LR) без стемминга
df4 = pd.DataFrame(classification_report(predicted_lr_l1, twenty_test_full.target, output_dict=True))

# Логистическая регрессия (LR) со стеммингом
df5 = pd.DataFrame(classification_report(predicted_lr_stem, twenty_test_full.target, output_dict=True))

# Логистическая регрессия_l1 (LR) со стеммингом
df6 = pd.DataFrame(classification_report(predicted_lr_l1_stem, twenty_test_full.target, output_dict=True))

# Метод опорных векторов (SVM) без стемминга
df7 = pd.DataFrame(classification_report(predicted_svm, twenty_test_full.target, output_dict=True))

# Метод опорных векторов (SVM) со стеммингом
df8 = pd.DataFrame(classification_report(predicted_svm_stem, twenty_test_full.target, output_dict=True))

df1.to_excel(writer, sheet_name='RF без стемминга')
df2.to_excel(writer, sheet_name='RF со стеммингом')

df3.to_excel(writer, sheet_name='LR без стемминга')
df4.to_excel(writer, sheet_name='LR_l1 без стемминга')

df5.to_excel(writer, sheet_name='LR со стеммингом')
df6.to_excel(writer, sheet_name='LR_l1 со стеммингом')

df7.to_excel(writer, sheet_name='SVM без стемминга')
df8.to_excel(writer, sheet_name='SVM со стеммингом')

writer.close()